In [ ]:
######################################################################################################
  ############ Updating bags_of_words, full word sums, and related articles lists ############ 
######################################################################################################

#####################################
 ##### TO DO FIRST TO MAKE THIS WORK: 
#####################################
########## 1. This is how you update allofplos: 
 # A. In powershell use: python -m allofplos.update
 # B. Not needed but runs faster: In the 'allofplos_xml' folder, sort by date modified and put all new files into the folder 'allofplos_temp'
########## 2. Change all xml files into txt: 
 # *** Go to the command prompt and type in:
   # A. cd\
   # B. cd\Users\jthibodeaux\python\Lib\site-packages\allofplos\allofplos_temp
   # C. ren *.xml *.txt



In [ ]:
#####################################
 ##### The below opens / reads all of the bag_of_words csvs from a folder into Python.
#####################################
import os
import pandas as pd
path = "final_allofplos_bagofwords/"
fileList = os.listdir(path)
bag_of_words = {}
dois = []
old_doi_list = []
for i in range(len(fileList)):
    bag_of_words[i] = pd.read_csv("final_allofplos_bagofwords/"+fileList[i])
    dois = bag_of_words[i].iloc[:,0]
    bag_of_words[i].index = dois
    old_doi_list.extend(dois)





#####################################
 ##### The below opens / reads all of the text files from a folder into Python.
#####################################
allLines = []
path = 'python/Lib/site-packages/allofplos/allofplos_temp/'
  # NOTE: This means the 'allofplos_temp' folder has to be a child folder of the folder of this file.
fileList = os.listdir(path)

####### Check if a new doi already exists in the old doi list. If so, then remove:
old_files = [i + ".txt" for i in old_doi_list]     # Add on the .txt to the old dois.
for i in old_files:
    if i in fileList:
        fileList.remove(i)

####### Create the new list of text:
for file in fileList:
   file = open(os.path.join('python/Lib/site-packages/allofplos/allofplos_temp/'+ file), 'r', encoding='utf-8')
   allLines.append(file.read())


####### This changes the file list into dois by taking off the .txt:
fileList = [x[:-4] for x in fileList]


#### Length of new file list (minus DOIs already in the existing bag_of_words):
len(allLines)




In [ ]:
#################################################
 ##### BOTH OF THE BELOW FILTERS BY RESEARCH-ARTICLES *AND* TAKES OUT HTML LANGUAGE
   ## NOTE: One reason I only get research articles is because NON-research articles (or some subset of them) have some weird issue with them when tokenizing
#################################################

#################################################
##### 1. The below use ALL WORDS from the article:
#################################################
############################################################################ Disabled for now
#import re
#from bs4 import BeautifulSoup
#full_corpus = []
#research_article_list = []
#for i in range(0, len(allLines)):
#    if len(re.findall('article-type="research-article', allLines[i])) > 0:    
#        soup = BeautifulSoup(allLines[i], "xml")
#        text = soup.get_text(strip=False)
#        full_corpus.append(text)
#        research_article_list.append(fileList[i])





#################################################
##### 2. The below removes all text except title and abstract:
#################################################

##################
######## This allows you to get the title and abstract: 
#### You use split to split the data by a particular string, then only use the string beforehand.
  # E.g. see: https://stackoverflow.com/questions/27387415/how-would-i-get-everything-before-a-in-a-string-python
##Title
   # Start: <title-group>
   # End: <alt-title alt-title-type
##Abstract_1:
   # Start: </permissions>
   # End: </abstract>
      # Also, add on a second group of text if there's over 1 </abstract> after the </permissions>
##Abstract_2:
   # Start: <abstract abstract-type="summary">
   # End: </abstract>
##################
import re
from bs4 import BeautifulSoup
full_corpus = []
research_article_list = []
for i in range(0, len(allLines)):
    if len(re.findall('article-type="research-article', allLines[i])) > 0:    
        text = str(allLines[i])
        ## Titles look to be called similarly across allofplos (abstracts are more difficult, see below)
        title = text.split('<alt-title alt-title-type')[0]
        title = title.split('<title-group>')[1]
        ## This is an additional way to include abstracts BUT they aren't in every article:
        Another_abstract = 0
        if len(re.findall('</abstract></article-meta>', text)) > 0:
            abstract_2 = text.split('</abstract></article-meta>')[0]
            abstract_2 = abstract_2.split('</permissions>')[1]
            Another_abstract = Another_abstract + 1
        else:         # This is the main way to get the abstract:
            abstract = text.split('</permissions>')[1]
            abstract_1 = abstract.split('</abstract>')[0]
        ### Get rid of xml tags for the title :
             ## NOTE: In many cases, using BeautifulSoup on the abstract portion removes a lot of the abstract so I disable it here (I remove related words in the below cell such as 'abstract':
        soup = BeautifulSoup(title, "xml")
        title = soup.get_text(strip=False)
        ### If there's more than one </abstract> after permissions, add the part after that first </abstract> too.
        if Another_abstract > 0:
            text = title + " " + abstract_2
        else:
            if len(re.findall('</abstract>', abstract)) > 1:   ## If there's more than one </abstract> after permissions, add the part after that first </abstract> too.
                abstract_3 = abstract.split('</abstract>')[1]
                text = title + " " + abstract_1 + " " + abstract_3
            else:
                text = title + " " + abstract_1
        full_corpus.append(text)
        research_article_list.append(fileList[i])


#### Length of new file list (minus DOIs already in the existing bag_of_words AND only including 'Research Articles'):
len(full_corpus)




In [ ]:
#####################################
 ##### Step 1: Create a loop to clean the articles (e.g. punctuation, upper cases) before you put it in the main method
  ## Make the string all lower case, take out numbers, take out punctuation, take out white spaces:
import nltk
import re
import string
## The following 2 could be skipped if you want to keep the stopwords:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

new_corpus = []
for i in range(len(full_corpus)):
    temp = []
    temp = full_corpus[i].lower()
    temp = re.sub(r'\d+', '', temp)
    temp = temp.translate(str.maketrans("", "", string.punctuation))
    temp = temp.strip()
    ### The following three could be skipped if you want to keep the stop words:
    temp = word_tokenize(temp)
    stop_words = set(stopwords.words("english"))
    temp = [i for i in temp if not i in stop_words]
    ### Get rid of any words you think should be removed (eg uninformative common words and tags)
    to_remove = ["plos", "use", "using", "abstract", "sec", "title", "italic", "summary", "type", "toc", "bold", "relatedarticle", "page", "relatedarticletype", "article-title", "sup", "abstracttype", "reftypebibr", "xref", "contenttypepdf", "ref", "idsec", "summarytitl", "pthe", "yearyear", "abstracttypesummari", "volumevolum", "xlinktypesimplenam", "mixedcit", "rowspan", "xlinktypesimpl", "italicin", "extlinktypeuri", "titleresultstitl", "titlemethodstitl", "fpagefpag", "abstracttypetoc", "italicsital", "lpagelpag", "cdsupsup", "id", "titlemethod", "metaanalysi", "sourceplo", "namedcont", "articletitleth", "mimetypeimag", "commentdoiextlink", "italicmital", "fpageefpag", "abstractsec", "titleresultstitl", "findingstitl", "titlemethodstitl", "italicpitaliclt", "titlemethod", "pthese", "iii", "persongroup", "tgfβ", "ip", "extlink']", "extlink", "fpagefpag", "mimetypeimag", "lpagelpag", "cellsp']", "cellsp", "secsec", "titleresultstitl", "titlemethodstitl", "italicpital", "sectypehead", "link", "sec", "titleobjectivetitl", "tlr", "findingstitl", "ital", "supplement", "namedcont", "italica", "titlemethod", "methodstitl", "italicsital", "titlepurposetitl", "italicd", "titlemateri", "italicnital", "italict", "italicnamedcont", "italich", "titletri", "italiceital", "contenttypegenu", "titleresultstitl", "titlemethodstitl", "italicpital", "xlinktypesimpl", "titleobjectivetitl", "italicsital", "italiccital", "italiceital", "subtyp", "italicaital", "italicbital", "italicnital", "titlepurposetitl", "titlemateri", "italiccoliital", "italiclital", "italicpital", "unique", "name", "use", "would", "either", "introduction"]
    ## Get rid of all words with 'link in them' (since beautiful soup doesn't work well on abstract):
    ## Get rid of all words with 'link in them' (since beautiful soup doesn't work well on abstract):
    bad_col_1 = re.compile("link:*")                       # NOTE: This matching on link bit is new.
    bad_col_2 = re.compile("abstract*")
    bad_col_3 = re.compile("title*")
    bad_col_4 = re.compile("xlink*")
    bad_col_5 = re.compile("label*")
    matches_1 = [i for i in temp if re.match(bad_col_1, i)]
    matches_2 = [i for i in temp if re.match(bad_col_2, i)]
    matches_3 = [i for i in temp if re.match(bad_col_3, i)]
    matches_4 = [i for i in temp if re.match(bad_col_4, i)]
    matches_5 = [i for i in temp if re.match(bad_col_5, i)]
    to_remove.extend(matches_1)
    to_remove.extend(matches_2)
    to_remove.extend(matches_3)
    to_remove.extend(matches_4)
    to_remove.extend(matches_5)
    temp = [i for i in temp if not i in to_remove]
    ### get rid of 1 and 2 letter words AND words with 20 characters or more:
    temp = [i for i in temp if (len(i) > 2) and (len(i) < 20) ]
    ### Stem the words (e.g. using and used becomes use)
    stemmer = PorterStemmer()
    temp = [stemmer.stem(i) for i in temp]
    new_corpus.append(str(temp))




#####################################
 ##### Step 2: Make a method to create a 'bag of words' (all unique words in the corpus and # of words in each article):
from collections import Counter
import pandas as pd

def make_matrix(corpus):
    matrix = []
    # Create a list of all the words in this chunk of the corpus:
    unique_words = []
    unique_words = list(set(" ".join(corpus).split()))
    for i in range(len(corpus)):
        # Get all unique words across all articles in this chunk:
        bbb = []
        bbb = corpus[i].split()
        counter = Counter(bbb)
        # Turn the dictionary into a matrix row using the vocab.
        row = [counter.get(w, 0) for w in unique_words]
        matrix.append(row)
    df = pd.DataFrame(matrix)
    # Make the column names the words
    df.columns = unique_words
    # Exclude columns with less than 3 words metioned in this chunk
    df.drop([col for col, val in df.sum().iteritems() if val < 3], axis=1, inplace=True)
    return df


new_corpus = make_matrix(new_corpus)





#####################################
 ##### Step 3: Bring in old bag_of_words sum of words in the entire corpus; update it and write to file:
#####################################
old_sum_columns = pd.read_csv("Full_allofplos_BagOfWords.csv")
new_sum_columns = new_corpus.sum(axis=0)

####### Left join the two dataframes:
tran_old_sum_columns = old_sum_columns.T
new_sum_columns = pd.DataFrame(new_corpus.sum(axis=0))
full_word_count = tran_old_sum_columns.join(new_sum_columns, lsuffix='_a', rsuffix='_b').T

####### Prepare dataset (fill NAs, sum all rows) and export as the revised main sum of all words: "Full_allofplos_bagofwords.csv"
full_word_count = full_word_count.fillna(0)
full_word_count = full_word_count.sum(axis=0)
full_word_count = pd.DataFrame(full_word_count)
full_word_count = full_word_count.T
full_word_count.to_csv(r"Full_allofplos_bagofwords.csv")






#####################################
 ##### Step 4: Add in rownames of the DOIs and add this bag_of_words to the big list of files:
#####################################
new_corpus = new_corpus.fillna(0)
new_corpus.index = research_article_list
#new_corpus.insert(0, "DOI to compare", research_article_list)

####### Add new bag of words to the rest of the csvs, then add onto the bag_of_words list of lists:
count = (len(bag_of_words))
new_corpus.to_csv(rf"final_allofplos_bagofwords/smaller_TFIDFs_dataset_{count}.csv")

bag_of_words[count] = new_corpus




In [ ]:
################################################################# 
 ##### Step 5: Use column sums of each dataframe to get final TF-IDF:
################################################################# 

####### First you need to take out the 1st column "Unnamed: 0" or 'DOI to compare', which contains the DOIs.
  # The index also has the DOI name so use index. 
###### Take out columns with strings in the values (would only be in first few columns but I went to 100 just in case):
for i in range(len(bag_of_words)):
    for j in range(100):
        if type(bag_of_words[i].iloc[1,j]) is str:
            bag_of_words[i] = bag_of_words[i].drop(bag_of_words[i].columns[j], axis=1)



#####################################
 ##### 5a. Get TF part of TF-IDF (values divided by their row sums)
for i in range(len(bag_of_words)):
    row_sum = bag_of_words[i].sum(axis=1)
    bag_of_words[i] = bag_of_words[i] / row_sum[:,None]





#####################################
 ##### 5b. Get IDF for all dataframes. 
##### Get IDF (column sums / sum of all words):
full_col_sum = pd.read_csv("Full_allofplos_BagOfWords.csv")

### Take out columns with the name 'Unnamed: X' (would only be in first few columns but I went up to 100 just in case):
import re
bad_col = re.compile("Unnamed:*")
for j in range(100):
    if len(bad_col.findall(full_col_sum.columns[j])) > 0:
        full_col_sum = full_col_sum.drop(full_col_sum.columns[j], axis=1)


### Transpose, get values, then sum.
sum_transposed = full_col_sum.T
sum_all_words = int(sum(sum_transposed.values))

idf = full_col_sum / sum_all_words


#####################################
 ##### 5c. Get TF-IDF for all dataframes: 
####### Divide all 'bag_of_words' (which is now TF: each value divided by it's row sum) by it's column's IDF:
#####################################
for i in range(len(bag_of_words)):
    ## Left join bag_of_words onto the full list of words: The makes the final row the corpus word count:
    bag_of_words[i] = bag_of_words[i].T
    trans_idf = idf.T
    bag_of_words[i] = bag_of_words[i].join(trans_idf).T
    ## Get TF-IDF for all rows; except final row (which is the sum of words in this loop)
       ## NOTE: This loop went quicker than the non-loop option: bag_of_words[i] = bag_of_words[i] / idf
    for j in range(len(bag_of_words[i].columns)):
        bag_of_words[i].iloc[:,j] = bag_of_words[i].iloc[:,j] / bag_of_words[i].iloc[(len(bag_of_words[i].index)-1),j]
    ## Drop the row with column sums:
    bag_of_words[i].drop(bag_of_words[i].tail(1).index,inplace=True)




In [ ]:
################################################################# 
 ##### Step 6: Use the TF-IDF to create one large dataframe where each DOI has n most relevant articles (in order)
################################################################# 

###### Combined with below, this allows you to time how fast your code runs:
import pandas as pd
import time
start_time = time.time()


##########################################
 ## MAIN:  Get top 19 DOIS by TF-IDF correlation between DOI being compared and all other DOIs in corpus.
    ### The below assumes the new bag_of_words is the final one in the list (which it's set up as in Step 4)
##########################################

recommendations = 20

final_data = pd.DataFrame(columns = range(recommendations))

for iter_doi in range(len(bag_of_words[(len(bag_of_words)-1)])):
    aa = pd.DataFrame(bag_of_words[(len(bag_of_words)-1)].iloc[iter_doi,:].T)
    # Reduce DOI to compare data to words that exist in that DOI:
    aa = aa.fillna(0)
    aa = aa[aa.values > 0]
    #Get dataframe of only TF-IDFs in DOI being compared over 20. 
    x =  aa[aa.values > 20]
    corrs = pd.DataFrame()
    # This loop gets the correlations of the DOI for each bag_of_words:
    for i in range(0, (len(bag_of_words)-1)):
        new = bag_of_words[i]
        # If 'new' is the same dataframe as the one that has the DOI being compared, remove the DOI being compared from 'new'
        if new.index[0] == bag_of_words[len(bag_of_words)-1].index[0]:
            new = new.drop([aa.columns[0]], axis = 0)
        #To increase speed, remove DOIs who have a summed TF-IDF of under 100 for TF-IDF over 5 in the DOI being compared.
        b = x.join(new.T)
        c = b.sum(axis=0)
        c = list(c[c.values < 1000].index)
        new = new.drop(c, axis=0)
        b = pd.concat([new, aa.T], sort=False)
        b = b.fillna(0).T
        #Do a corelation of the first dataframe with the second:
        c = pd.DataFrame(b[b.columns[0:]].corr()[b.columns[len(b.columns)-1]])[:-1]
        #Cut down to the number of recommendation and format to prepare for final data:
        corrs = pd.concat([corrs, c], sort=False)
        corrs = corrs.sort_values(corrs.columns[0], ascending=False).head(n = recommendations)
    to_compare = [corrs.columns[0]]
    corrs = pd.DataFrame(corrs.index)
    corrs.columns = to_compare
    corrs = corrs.T
    final_data = pd.concat([final_data, corrs], sort=False)
    ## NOTE: I put this in the loop becasue this takes so long, enables starting and stopping.
    final_data.to_csv(rf"RelatedDoiList_Best_Correlations.csv")

final_data.index = pd.Index(final_data.index, name = 'DOI Being Compared')   # use if index is the column of the compared DOI


###### Combined with above, this allows you to time how fast your code runs:
print("My program took", time.time() - start_time, "to run")




In [ ]:
################################################################# 
 ##### Step 7: The below inserts new related DOIs into the existing related dois list:
    ## E.g. DOI X is the 5th most related article for a new DOI Y. Using the below, DOI X will now have DOI Y as their 5th most related article, the existing 5th through 2nd to last related articles are moved down, and the least related article in the list is removed.
    ## NOTE: It is now time optimized within the loop by only searching through rows in the old list that will be changed for each new DOI.
      ## Takes about 0.8 seconds per new DOI
#################################################################

import time
start_time = time.time()


######## This is where I bring in the existing DOI related articles list and new DOI related articles list.
  ## If these are already in the memory of the Jupyter notebook you could change the code or for convenience just do: new = final_data.copy()
import pandas as pd
existing = pd.read_csv("RelatedDoiList_Best_Correlations_ALL.csv")
new = pd.read_csv("RelatedDoiList_Best_Correlations.csv")
#### Or import new from the cell above:
#new = final_data.copy()

######## Attach new to existing first (so new DOIs can be updated with new DOI matches too):
new = new.rename(columns={new.columns[0]: "DOI Being Compared"})
existing = pd.concat([existing, new], sort = False)


######## This is the main loop to insert new DOIs into the old:
for new_row in range(len(new)):
    ###### Subset for each new_col and new_row to significantly speed up loop:
    ## 1. Make a list of related DOIs in the current row of new:
    new_doi_list = list(new.iloc[new_row,1:len(new.columns)])
    rows_to_modify_list = []
    for i in range(1, len(existing.columns)-1):   ### Skip the last column ... If need be for time, you could skip the last n columns (e.g. skipping 17-19 may save a good amount of time if you're updating over half the rows in the dataset) ... If you do so, change the line (37) with "Skip the last column" below similarly. 
        rows_to_modify_list.extend(list(existing[existing.iloc[:,i].isin(new_doi_list)].iloc[:,0]))
    ## 3. Create two datasets: One with rows that will not be changed, the other with rows that will be changed:
    unchanged_data = existing[~existing.iloc[:,0].isin(rows_to_modify_list)]
    changed_data = existing[existing.iloc[:,0].isin(rows_to_modify_list)].copy()
    for new_col in range(1, (len(new.columns) - 2)):  ### Skip the last column
        for exist_row in range(len(changed_data)):
            if new.iloc[new_row, new_col] == changed_data.iloc[exist_row, 0]:   ### NOTE: The 'existing column' is always 0 here.
                ## 1. Replace the 2nd through last column's values with the 1st through 2nd to last column values.
                changed_data.iloc[exist_row, (new_col+1):(len(changed_data.columns))] = list(changed_data.iloc[exist_row, new_col:(len(changed_data.columns)-1)])
                ## 2. Replace the first value with the 1st value from the new dataframe.
                changed_data.iloc[exist_row, new_col] = new.iloc[new_row, 0]
    ###### Change 'existing' by concatinating changed data with unchanged data:
    existing = pd.concat([unchanged_data, changed_data])


######## Export to CSV file:
existing.to_csv(r"RelatedDoiList_Best_Correlations_ALL.csv", index=False)

print("My program took", time.time() - start_time, "to run")

